# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

622

In [3]:
url_weather="http://api.openweathermap.org/data/2.5/weather?"
url=f"{url_weather}appid={weather_api_key}&q={city}"

response=requests.get(url)
data=response.json()
data

{'coord': {'lon': 94.5, 'lat': 25.6667},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 295.66,
  'feels_like': 292.31,
  'temp_min': 295.66,
  'temp_max': 295.66,
  'pressure': 1010,
  'humidity': 24,
  'sea_level': 1010,
  'grnd_level': 834},
 'visibility': 10000,
 'wind': {'speed': 2.15, 'deg': 288},
 'clouds': {'all': 35},
 'dt': 1615364481,
 'sys': {'country': 'IN', 'sunrise': 1615334201, 'sunset': 1615376891},
 'timezone': 19800,
 'id': 1259784,
 'name': 'Phek',
 'cod': 200}

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
#checking for the URL query

count_city=1

city_data=[]
coord_lat=[]
coord_lon=[]
main_max_temp=[]
humidity=[]
cloudiness=[]
wind_speed=[]
country=[]
dt=[]

for city in cities:
    
    #retrive url and request the json file
    url_weather="http://api.openweathermap.org/data/2.5/weather?"
    query_url=f"{url_weather}appid={weather_api_key}&q={city}"
    weather_response=requests.get(query_url)
    weather_json=weather_response.json()
    
    #run loop on all cities with data needed.
    try:
        
        coord_lat.append(weather_json['coord']['lat'])
        coord_lon.append(weather_json['coord']['lon'])
        main_max_temp.append(weather_json['main']['temp_max'])
        humidity.append(weather_json['main']['humidity'])
        cloudiness.append(weather_json['clouds']['all'])
        wind_speed.append(weather_json['wind']['speed'])
        country.append(weather_json['sys']['country'])
        dt.append(weather_json['dt'])
        city_data.append(city)
        print(f"Processing Record {count_city}|{city}")
    except:
         print(f'City not found. Skipping...{city}')
            
    count_city+=1

    

Processing Record 1|manokwari
Processing Record 2|mahebourg
Processing Record 3|hithadhoo
City not found. Skipping...vaitupu
City not found. Skipping...belushya guba
Processing Record 6|busselton
Processing Record 7|chipinge
Processing Record 8|fortuna
Processing Record 9|pisco
Processing Record 10|puerto ayora
Processing Record 11|te anau
Processing Record 12|ribeira grande
Processing Record 13|burns lake
Processing Record 14|vaini
City not found. Skipping...nizhneyansk
Processing Record 16|bluff
Processing Record 17|bud
Processing Record 18|mataura
Processing Record 19|karratha
City not found. Skipping...amderma
Processing Record 21|sola
Processing Record 22|kaeo
Processing Record 23|hilo
Processing Record 24|lavrentiya
Processing Record 25|guadalajara
Processing Record 26|kanniyakumari
Processing Record 27|harper
Processing Record 28|ambulu
Processing Record 29|bathsheba
City not found. Skipping...illoqqortoormiut
Processing Record 31|narsaq
Processing Record 32|mar del plata
Proces

KeyboardInterrupt: 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [8]:


city_dic={'City':city_data,
          'Lat':coord_lat,
          'lng':coord_lon,
          'Max Temp':main_max_temp,
          'Humidity':humidity,
          'Cloudiness':cloudiness,
          'Wind Speed':wind_speed,
          'Country':country,
          'Date':dt}
city_data_df=pd.DataFrame(city_dic)
city_data_df

#export the city data into .CSV
city_data_df.to_csv('weather_CSV/weather_data.csv')

#display the data frame.
city_data_df

City      Lat       lng  Max Temp  Humidity  Cloudiness  \
0               manokwari  -0.8667  134.0833    299.89        77          87   
1               mahebourg -20.4081   57.7000    303.15        66          40   
2               hithadhoo  -0.6000   73.0833    302.13        63           4   
3               busselton -33.6500  115.3333    303.71        55          91   
4                chipinge -20.1883   32.6236    298.28        57           0   
..                    ...      ...       ...       ...       ...         ...   
392  vitoria da conquista -14.8661  -40.8394    290.32        91          29   
393             marquette  46.5333  -87.6335    280.93        70         100   
394          spas-demensk  54.4122   34.0226    261.93        95         100   
395               wasilla  61.5814 -149.4394    273.15        93          90   
396               ojinaga  29.5667 -104.4167    291.15        36           1   

     Wind Speed Country        Date  
0          3.11      ID  1615364481  
1          5.66      MU  1615364481  
2          2.59      MV  1615364482  
3          1.79      AU  1615364482  
4          1.32      ZW  1615364483  
..          ...     ...         ...  
392        1.50      BR  1615364410  
393        0.89      US  1615364599  
394        2.43      RU  1615364599  
395        1.76      US  1615364600  
396        2.57      MX  1615364354  

[397 rows x 9 columns]

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression